# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# Import necessary libraries
import os
import numpy as np
import pandas as pd
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img, img_to_array
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os 
import numpy as np 
import pandas as pd 
import tensorflow as tf 
from keras.applications.vgg16 import VGG16 
from keras.preprocessing.image import load_img, img_to_array
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from sklearn.model_selection import train_test_split

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

2024-05-31 12:54:56.081737: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-31 12:54:56.081890: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-31 12:54:56.272145: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/sample_submission.csv
/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_series_descriptions.csv
/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train.csv
/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_label_coordinates.csv
/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_series_descriptions.csv
/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/1737682527/2291122880/12.dcm
/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/1737682527/2291122880/18.dcm
/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/1737682527/2291122880/9.dcm
/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/1737682527/2291122880/14.dcm
/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/1737682527/2291122880/11.dcm
/kaggle/input/rsna-202

In [17]:
# Load datasets
train_df = pd.read_csv('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train.csv')
label_coords = pd.read_csv('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_label_coordinates.csv')
train_series_desc = pd.read_csv('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_series_descriptions.csv')

In [39]:
# Inspect columns of each DataFrame
print("train_df columns:", train_df.columns)
print("label_coords columns:", label_coords.columns)
print("train_series_desc columns:", train_series_desc.columns)

train_df columns: Index(['study_id', 'spinal_canal_stenosis_l1_l2',
       'spinal_canal_stenosis_l2_l3', 'spinal_canal_stenosis_l3_l4',
       'spinal_canal_stenosis_l4_l5', 'spinal_canal_stenosis_l5_s1',
       'left_neural_foraminal_narrowing_l1_l2',
       'left_neural_foraminal_narrowing_l2_l3',
       'left_neural_foraminal_narrowing_l3_l4',
       'left_neural_foraminal_narrowing_l4_l5',
       'left_neural_foraminal_narrowing_l5_s1',
       'right_neural_foraminal_narrowing_l1_l2',
       'right_neural_foraminal_narrowing_l2_l3',
       'right_neural_foraminal_narrowing_l3_l4',
       'right_neural_foraminal_narrowing_l4_l5',
       'right_neural_foraminal_narrowing_l5_s1',
       'left_subarticular_stenosis_l1_l2', 'left_subarticular_stenosis_l2_l3',
       'left_subarticular_stenosis_l3_l4', 'left_subarticular_stenosis_l4_l5',
       'left_subarticular_stenosis_l5_s1', 'right_subarticular_stenosis_l1_l2',
       'right_subarticular_stenosis_l2_l3',
       'right_subarticular_

In [40]:
# Merge dataframes for easier handling
# Ensure 'series_id' exists in both DataFrames for merging
if 'series_id' in train_df.columns and 'series_id' in label_coords.columns:
    train_df = train_df.merge(label_coords, on=['study_id', 'series_id'], how='left')
else:
    train_df = train_df.merge(label_coords, on='study_id', how='left')

if 'series_id' in train_df.columns and 'series_id' in train_series_desc.columns:
    train_df = train_df.merge(train_series_desc, on=['study_id', 'series_id'], how='left')
else:
    train_df = train_df.merge(train_series_desc, on='study_id', how='left')

print(train_df.head())


   study_id spinal_canal_stenosis_l1_l2 spinal_canal_stenosis_l2_l3  \
0   4003253                 Normal/Mild                 Normal/Mild   
1   4003253                 Normal/Mild                 Normal/Mild   
2   4003253                 Normal/Mild                 Normal/Mild   
3   4003253                 Normal/Mild                 Normal/Mild   
4   4003253                 Normal/Mild                 Normal/Mild   

  spinal_canal_stenosis_l3_l4 spinal_canal_stenosis_l4_l5  \
0                 Normal/Mild                 Normal/Mild   
1                 Normal/Mild                 Normal/Mild   
2                 Normal/Mild                 Normal/Mild   
3                 Normal/Mild                 Normal/Mild   
4                 Normal/Mild                 Normal/Mild   

  spinal_canal_stenosis_l5_s1 left_neural_foraminal_narrowing_l1_l2  \
0                 Normal/Mild                           Normal/Mild   
1                 Normal/Mild                           Normal/M

In [41]:
# Define conditions and levels
conditions = ['spinal_canal_stenosis', 'neural_foraminal_narrowing', 'subarticular_stenosis']
levels = ['l1_l2', 'l2_l3', 'l3_l4', 'l4_l5', 'l5_s1']
for condition in conditions:
    for level in levels:
        col = f'{condition}_{level}'
        if col not in train_df.columns:
            train_df[col] = np.nan

In [43]:
# Convert all condition columns to string dtype to handle 'Unknown' values
for condition in conditions:
    for level in levels:
        col = f'{condition}_{level}'
        train_df[col] = train_df[col].astype('str')

In [44]:
# Handle missing labels
train_df.fillna('Unknown', inplace=True)
print(train_df.head())

/tmp/ipykernel_33/415608190.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Unknown' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  train_df.fillna('Unknown', inplace=True)


   study_id spinal_canal_stenosis_l1_l2 spinal_canal_stenosis_l2_l3  \
0   4003253                 Normal/Mild                 Normal/Mild   
1   4003253                 Normal/Mild                 Normal/Mild   
2   4003253                 Normal/Mild                 Normal/Mild   
3   4003253                 Normal/Mild                 Normal/Mild   
4   4003253                 Normal/Mild                 Normal/Mild   

  spinal_canal_stenosis_l3_l4 spinal_canal_stenosis_l4_l5  \
0                 Normal/Mild                 Normal/Mild   
1                 Normal/Mild                 Normal/Mild   
2                 Normal/Mild                 Normal/Mild   
3                 Normal/Mild                 Normal/Mild   
4                 Normal/Mild                 Normal/Mild   

  spinal_canal_stenosis_l5_s1 left_neural_foraminal_narrowing_l1_l2  \
0                 Normal/Mild                           Normal/Mild   
1                 Normal/Mild                           Normal/M

In [45]:
# Load VGG16 model (pre-trained on ImageNet)
base_model = VGG16(weights="imagenet", include_top=False)

In [46]:
# Function to extract features from images
def extract_features(image_path):
    img = load_img(image_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    features = base_model.predict(img_array)
    return features.flatten()

In [48]:
#'train_images' is the folder containing images
image_folder = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images'

In [51]:
# Prepare features (X) and labels (y)
X = []
y = []
for idx, row in train_df.iterrows():
    image_path = os.path.join(image_folder, str(row['study_id']), str(row['series_id']), f"{row['instance_number']}.dcm")
    if os.path.exists(image_path):
        features = extract_features(image_path)
        X.append(features)
        # Combine condition and level columns to form the labels
        labels = [row[f'{condition}_{level}'] for condition in conditions for level in levels]
        y.append(labels)

X = np.array(X)
y = np.array(y)


KeyError: 'instance_number'